### Tests GPU

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Load data

In [2]:
from utils.process_tensors import load_tensor, normalize_tensor
import tensorflow as tf

# Parameters
loss = 'mse'
nb_layers = 3
nb_filters = 32
batch_size = 16
shape = (8,8,8)

train_ratio = .8

file_names = ['test1']

input_tensor = None
output_tensor = None
for file_name in file_names:
    load_path = f"./data/{file_name}/"
    input_path = load_path + "input"
    output_path = load_path + "output"
    x = load_tensor(input_path, out_type=tf.int32)
    y = load_tensor(output_path, out_type=tf.float32)
    if input_tensor is None:
        input_tensor = x
        output_tensor = y
    else:
        input_tensor = tf.concat([input_tensor, x], axis=0)
        output_tensor = tf.concat([output_tensor, y], axis=0)




### Transform Data

In [3]:
from utils.process_tensors import transform_square_root_tensor, reverse_transform_square_root_tensor

max_value = tf.reduce_max(tf.abs(output_tensor)).numpy()
transformed_tensor = transform_square_root_tensor(output_tensor, max_value=1.0)


### Visualize data

In [4]:
# import matplotlib.pyplot as plt

# fig, axs = plt.subplots(1, 2, figsize=(20, 2.5))
# bins = 50
# axs[0].hist(output_tensor, bins=bins)
# axs[1].hist(transformed_tensor, bins=bins)
# print(f'max value of output_tensor: {max_value}')
# print histogram of output_array


### Make Dataset + separate into train and test

In [5]:
dataset:tf.data.Dataset = tf.data.Dataset.from_tensor_slices((input_tensor, transformed_tensor))
print('Number of positions:',dataset.cardinality().numpy())

# batch and shuffle
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(1)
dataset = dataset.shuffle(buffer_size=dataset.cardinality().numpy())

# split
train_size = int(dataset.cardinality().numpy() * train_ratio)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

print('Number of batches:',batch_size)
print('train size:',train_dataset.cardinality().numpy())
print('test size:',test_dataset.cardinality().numpy())

Number of positions: 10000
Number of batches: 16
train size: 500
test size: 125


### Model

In [6]:
from utils.build_model import build_model

model = build_model(nb_filters=nb_filters , nb_layers=nb_layers, shape=shape)
model.compile(optimizer='adam', loss=loss, metrics=['mae', 'mse', 'msle'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8, 8, 32)          2336      
                                                                 
 dropout (Dropout)           (None, 8, 8, 32)          0         
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 32)          9248      
                                                                 
 dropout_1 (Dropout)         (None, 8, 8, 32)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 32)          9248      
                                                                 
 dropout_2 (Dropout)         (None, 8, 8, 32)          0         
                                                                 
 flatten (Flatten)           (None, 2048)              0

In [7]:
import os
from utils.callback_setup import callback_setup, create_checkpoint_name
# checkpoint_name = f"{loss}_{nb_layers}_{nb_filters}_{batch_size}_02/"

checkpoint_name = create_checkpoint_name(loss=loss, nb_layers=nb_layers, nb_filters=nb_filters, batchsize=batch_size, shape=shape, try_nb=1)
print('checkpoint name:',checkpoint_name)
logs_dir = f'logs/{checkpoint_name}/'
checkpoint_dir = os.path.join('backup', checkpoint_name)

model_callbacks = callback_setup(checkpoint_dir, logs_dir, monitor='loss', mode='min')

checkpoint name: loss=mse-layers=3-filters=32-batchsize=16-shape=888-try=01


### Load Weights

In [8]:
#checkpoint_name = "test_3_layers_4/"
try:
    checkpoint_filepath = os.path.join('backup', checkpoint_name)

    model.load_weights(checkpoint_filepath)
except:
    print("No checkpoint found.")
    pass

In [31]:
history = model.fit(train_dataset,
                    epochs=100,
                    validation_data=(test_dataset),
                    verbose=1,
                    callbacks=model_callbacks
                    )

Epoch 1/100


500/500 [==============================] - 17s 12ms/step - loss: 0.0812 - mae: 0.1850 - mse: 0.0812 - msle: 0.0203 - val_loss: 0.0531 - val_mae: 0.1499 - val_mse: 0.0531 - val_msle: 0.0151
Epoch 2/100
500/500 [==============================] - 6s 13ms/step - loss: 0.0590 - mae: 0.1591 - mse: 0.0590 - msle: 0.0147 - val_loss: 0.0464 - val_mae: 0.1418 - val_mse: 0.0464 - val_msle: 0.0129
Epoch 3/100
500/500 [==============================] - 5s 10ms/step - loss: 0.0532 - mae: 0.1516 - mse: 0.0532 - msle: 0.0131 - val_loss: 0.0397 - val_mae: 0.1350 - val_mse: 0.0397 - val_msle: 0.0112
Epoch 4/100
500/500 [==============================] - 6s 11ms/step - loss: 0.0511 - mae: 0.1480 - mse: 0.0511 - msle: 0.0126 - val_loss: 0.0533 - val_mae: 0.1482 - val_mse: 0.0533 - val_msle: 0.0158
Epoch 5/100
500/500 [==============================] - 6s 12ms/step - loss: 0.0499 - mae: 0.1459 - mse: 0.0499 - msle: 0.0125 - val_loss: 0.0467 - val_mae: 0.1400 - val_mse: 0.0467 - val_msle: 0.0126
Epoch 6/100

### Test on data from the training set

In [33]:
from utils.process_tensors import reverse_transform_square_root_tensor
import random

k = random.randrange(0, 1000)
n = 1

x_test = input_tensor[k:k+n,:,:,:]
y_test = transformed_tensor[k:k+n]

y_pred = model.predict(x_test, verbose=0)

print("---- With transformation ----")
print("true score: ", y_test.numpy())
print("predicted score: ", y_pred[:,0])

print("\n---- Real score ----")
print("true score: ", reverse_transform_square_root_tensor(y_test, max_value).numpy())
print("predicted score: ", reverse_transform_square_root_tensor(y_pred[:,0], max_value).numpy())

---- With transformation ----
true score:  [-0.03605732]
predicted score:  [-0.06413081]

---- Real score ----
true score:  [-0.13]
predicted score:  [-0.4112349]


### Tests on random board

In [51]:
from utils.random_board import generate_random_board
from utils.process_board import convert_board_to_array
from utils.stockfish import get_score

board = generate_random_board()
score = get_score(board)
x_test = tf.convert_to_tensor(convert_board_to_array(board, shape=shape))
x_test = tf.expand_dims(x_test, axis=0)

y_pred = model.predict(x_test)

print("predicted transformed score: ", y_pred[0][0])

print("\nTrue score: ", score)
print("Predicted score: ", reverse_transform_square_root_tensor(y_pred, max_value).numpy()[0][0])


1/1 [==============================] - 0s 18ms/step
predicted transformed score:  -0.19987708

True score:  5.22
Predicted score:  -3.9946852
